## Load Data

In [1]:
from src.utils import load_replay_buffer_and_kernel

FOLDER_NAME: str="data/local/experiment/circle_rotation"
TASK_NAMES=["sac_circle_rotation_task_0", "sac_circle_rotation_task_1", "sac_circle_rotation_task_2", "sac_circle_rotation_task_3"]

LOAD_WHAT:str="next_observations"
KERNEL_DIM=1
N_SAMPLES=50_000

tasks_ps, tasks_frameestimators=[], []
for task_name in TASK_NAMES:
    ps, frameestimator = load_replay_buffer_and_kernel(task_name, LOAD_WHAT, KERNEL_DIM, N_SAMPLES, FOLDER_NAME)
    tasks_ps.append(ps)
    tasks_frameestimators.append(frameestimator)

2025-07-13 17:06:51.538624: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/paulnitschke/Desktop/projects/geo_meta_rl/src/utils.py:100: UserWarning: Replay buffer contains more samples than selected.
  warnings.warn("Replay buffer contains more samples than selected.")


Loaded next_observations from data/local/experiment/circle_rotation/sac_circle_rotation_task_0_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation.


Loaded next_observations from data/local/experiment/circle_rotation/sac_circle_rotation_task_1_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation.


Loaded next_observations from data/local/experiment/circle_rotation/sac_circle_rotation_task_2_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation.


Loaded next_observations from data/local/experiment/circle_rotation/sac_circle_rotation_task_3_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation.


In [26]:
import wandb
%load_ext autoreload
%autoreload 2

import torch
from src.learning.symmetry.hereditary_geometry_discovery import HereditaryGeometryDiscovery
from src.utils import Affine2D


train_goal_locations=[
    {'goal': torch.tensor([-0.70506063,  0.70914702])},
 {'goal': torch.tensor([ 0.95243384, -0.30474544])},
 {'goal': torch.tensor([-0.11289421, -0.99360701])},
 {'goal': torch.tensor([-0.81394263, -0.58094525])}]

SEED=42
LEARN_LEFT_ACTIONS=True
LEARN_GENERATOR=True
LEARN_ENCODER_DECODER=False
USE_ORACLE_ROTATION_KERNEL=True
N_STEPS=100_000
BATCH_SIZE=128
BANDWIDTH=None

ENCODER=Affine2D(input_dim=2, output_dim=2)
DECODER=Affine2D(input_dim=2, output_dim=2)
ORACLE_GENERATOR=torch.tensor([[0, -1], [1,0]], dtype=torch.float32, requires_grad=False).unsqueeze(0) if not LEARN_GENERATOR else None

WAND_PROJECT_NAME="circle_hereditary_geometry_discovery"

wandb.init(project=WAND_PROJECT_NAME, config={
    "n_steps": N_STEPS,
    "batch_size": BATCH_SIZE,
    "kernel_dim": KERNEL_DIM,
    "bandwidth": BANDWIDTH,
    "learn_encoder_decoder": LEARN_ENCODER_DECODER,
    "learn_left_actions": LEARN_LEFT_ACTIONS,
    "learn_generator": LEARN_GENERATOR,
    "seed": SEED,
    "use_oracle_rotation_kernel": USE_ORACLE_ROTATION_KERNEL,
})

her_geo_dis=HereditaryGeometryDiscovery(tasks_ps=tasks_ps,
                                        tasks_frameestimators=tasks_frameestimators, 
                                        kernel_dim=KERNEL_DIM, 
                                        batch_size=BATCH_SIZE, 
                                        seed=SEED, 
                                        bandwidth=BANDWIDTH,
                                        log_wandb=True,
                                        learn_encoder_decoder=LEARN_ENCODER_DECODER,
                                        use_oracle_rotation_kernel=USE_ORACLE_ROTATION_KERNEL,
                                        task_specifications=train_goal_locations,
                                        learn_left_actions=LEARN_LEFT_ACTIONS,
                                        learn_generator=LEARN_GENERATOR,
                                        oracle_generator=ORACLE_GENERATOR,
                                        encoder=ENCODER,
                                        decoder=DECODER)
her_geo_dis.optimize(n_steps=N_STEPS)
wandb.finish()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


init/log_left_actions,█▆▅▅▃▂▂▂▂▂▂▂▂▂▇▂▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁
train/generator,██▇▇▆▅▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/left_actions/mean,█▆▄▃▄▄▃▃▂▁▃▂▁▃▂▁▂▁▁▁▂▁▁▁▁▂▁▁▁▁▂▂▁▁▂▂▂▁▂▂
train/regularizers/left_actions/lasso,█▆▅▅▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/regularizers/symmetry,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/symmetry/reconstruction,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/symmetry/span,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
init/log_left_actions,0.09191
train/generator,0.62882
train/left_actions/mean,0.29155
train/regularizers/left_actions/lasso,0.64866


INFO:root:Fitting left-actions: True
INFO:root:Fitting generator: True
INFO:root:Fitting encoder and decoder: False
INFO:root:Fitting log-linear regressors to initialize left actions.
INFO:root:Finished fitting log-linear regressors to initialize left actions.
Inferring left-action:   0%|          | 0/100000 [00:00<?, ?it/s]/Users/paulnitschke/Desktop/projects/geo_meta_rl/src/learning/symmetry/hereditary_geometry_discovery.py:161: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  goals = torch.stack([torch.tensor(self.task_specifications[i]['goal']) for i in self.task_idxs])
Left-Action Loss: 0.0 | Task Losses: [0. 0. 0.] | Task Losses (reg): 1.0529999732971191 | Generator Span Loss: 0.0:   7%|▋         | 7351/100000 [00:41<09:00, 171.46it/s]             

In [19]:
her_geo_dis.generator

Parameter containing:
tensor([[[ 1.6041e-04, -1.7604e+00],
         [ 5.6741e-01, -4.3644e-06]]], requires_grad=True)

In [17]:
her_geo_dis.lgs

tensor([[[ 9.9982e-01,  1.8022e-05],
         [-1.7940e+00,  1.0024e+00]],

        [[-7.4284e-01, -1.1793e+00],
         [ 3.8006e-01, -7.4292e-01]],

        [[ 9.9992e-01, -1.6791e+00],
         [-2.7793e-05,  1.0000e+00]]], grad_fn=<LinalgMatrixExpBackward0>)

In [18]:
for param in her_geo_dis.encoder.parameters():
    print(param.data)

tensor([[ 1.1399,  0.1600],
        [-0.0908,  0.6472]])
tensor([ 0.6903, -0.5230])
